## Useful Tips

In this section, we discuss some of the useful mechanisms in simulus to help accomplish modeling tasks.

### Pseudo-Random Numbers

So far we have been using Python's `random` module to generate all the needed pseudo-random numbers in our examples. One can also use `numpy.random` to generate random numbers. Both generators use Mersenne twister, which has very good random properties. Using `numpy.random` has a couple of advantages: one is that it provides more random distributions; the other is that the numpy module can be used to generate numpy arrays vert efficiently. Of course, using `numpy.random` requires numpy to be installed. 

The following shows the single-server queue example which we discussed earlier, but this time, we use different random distributions for the inter-arrival time and the service time. We use a Pareto distribution for the arrivals and a Gamma distribution for services.

In [ ]:
import numpy # assuming numpy has been installed
import simulus

numpy.random.seed(123)

def job(idx):
    r.acquire()
    print("%g: job(%d) gains access" % (sim.now,idx))
    sim.sleep(numpy.random.gamma(2, 2))
    print("%g: job(%d) releases" % (sim.now,idx))
    r.release()

def arrival():
    i = 0
    while True:
        i += 1
        sim.sleep(numpy.random.pareto(0.95))
        print("%g: job(%d) arrives" % (sim.now,i))
        sim.process(job, i)

sim = simulus.simulator()
r = sim.resource()
sim.process(arrival)
sim.run(10)


Actually, all our examples use the module's default random generator (`random` or `numpy.random`). All the functions supplied by the module are actually bound methods of a hidden instance of the corresponding class (`random.Random` or `numpy.random.RandomState`). Although it's OK for all the examples we have shown, there is a slight problem if we use the module-bound random generator:
* When we instantiate more than one simulators, they will share the random sequence. This means the simulation result may be different from running each simulator alone. This could make debugging difficult.
* When we run parallel and distributed simulation (as we will discuss later), if all simulation instances start with the same random seed, they would use the same random sequence. This is not a good idea because this would artificially introduce correlations among supposedly independent random processes.

Simulus provides an easy way to produce an independent random sequence at each simulator. The simulator's `rng()` method returns a random number generator that's attached to the simulator. There is one and only one such random number generator at each simulator. Internally, the generator is created with a seed taken from the `random` module's default random sequence (which is supposed to be seeded with the same constant), and the name of the simulator (which is supposed to be uniquely chosen by the user, or by simulus using the default random sequence if the simulator is created anonymously). Consequently, the random sequence will be consistant (across different runs and on differnt parallel machines) and guaranteed to be independent among simulators, as long as we use the same random seed to start the simulation. 

In [8]:
# %load "../examples/misc/mm1-numpy.py"
import numpy # assuming numpy has been installed
import random, simulus

# this determines the random module's default random sequence
random.seed(123)

def job(idx):
    r.acquire()
    print("%g: job(%d) gains access" % (sim.now,idx))
    sim.sleep(rng.gamma(2, 2))
    print("%g: job(%d) releases" % (sim.now,idx))
    r.release()

def arrival():
    i = 0
    while True:
        i += 1
        sim.sleep(rng.pareto(0.95))
        print("%g: job(%d) arrives" % (sim.now,i))
        sim.process(job, i)

sim = simulus.simulator('unique_name')
rng = numpy.random.RandomState(sim.rng().randrange(2**32))
r = sim.resource()
sim.process(arrival)
sim.run(10)


0.295163: job(1) arrives
0.295163: job(1) gains access
1.84229: job(1) releases
7.29799: job(2) arrives
7.29799: job(2) gains access
7.30716: job(3) arrives
8.62792: job(2) releases
8.62792: job(3) gains access
8.80694: job(3) releases
9.95584: job(4) arrives
9.95584: job(4) gains access
9.96089: job(5) arrives


In the above example, we create a numpy random number generator `rng` and seed it using the simulator's attached generator. The simulator is also named. In this case, the random number sequence from `rng` is going to be dependent only on the initial random seed 123, regardless whether the simulator is running together with other simulators or as part of a parallel simulation on different ranks. 

### Statistical Data Collection

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt